In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
sns.set_style("whitegrid")

In [2]:
df = pd.read_csv("merged_2021-Aug-21_1530.csv")
df_reg = pd.read_csv("Event_Registration.csv")

In [3]:
print('Size before deleating strings without word stimulus:', df.shape, sep = ' ')
df_clean = df[~df.word.isna()]
df_clean.reset_index(inplace=True)
print('Size after deleating them and adding the column with indexes:', df_clean.shape, sep = ' ')

Size before deleating strings without word stimulus: (11730, 9)
Size after deleating them and adding the column with indexes: (2340, 10)


In [4]:

# manually enter
target_array = [2,2,1,1,1,1,1,4,4,2,1,1,1,3,3,4,4,2,1,1,1,1,4,3,3,2,1,3,3,1]#participants' group numbers
n_word = len(df_clean['word'].unique())
n_odor = len(df_clean['odor'].unique())
n_exp = n_odor*n_word

In [5]:
df_clean.insert(7, column="target", value=np.repeat(np.array(target_array), n_exp))

In [6]:
# finds participants with latency times greater than 250 ms in at least a quarter of responses
extra_lat = list(df[df.latency < 0.25].participant)
extra_dict = Counter(extra_lat)
delete_part = []
for key, value in extra_dict.items():
    if value > n_exp/4:
        delete_part.append(key)
        
print(delete_part)

['008-1', '26.0']


In [7]:
for i in delete_part:
    df_clean = df_clean[df_clean.participant != i]
print('Now we have ' + str(int(df_clean.shape[0]/n_exp)) + ' participants' )

Now we have 28 participants


In [8]:
latency_norm = np.zeros(df_clean.shape[0])
for i in range(int(df_clean.shape[0]/n_exp)):
    sliced = df_clean['latency'][n_exp * i:n_exp * (i + 1)]
    latency_norm[n_exp * i:n_exp * (i + 1)] = (sliced - sliced.min()) / sliced.std()

In [9]:
df_clean.insert(4, column="latency_normed", value=latency_norm)
df_clean['comb'] = df_clean['odor'] + df_clean['word']

In [10]:
df_clean.to_csv('out.csv')